In [22]:
import os
import numpy as np, pandas as pd, matplotlib.pyplot as plt
%matplotlib inline
file_dir = os.path.dirname(os.getcwd())
labels = ('popularity', 'BPM', 'energy', 'danceability',
          'loudness', 'liveness', 'valence', 'length',
          'acousticness', 'speechiness')
data = pd.read_csv(file_dir + '/spotify_data.csv', names = labels)
features = data.values[:, 1:]
target = data.values[:,0]
n, m = features.shape[0], features.shape[1] + 1
X = np.empty((n, m))
X[:, 0] = np.ones(n)
X[:, 1:] = features

In [23]:
class OLS: 

    def __init__(self):
        pass

    def fit(self, X, y):
        self.X, self.y = X, y
        self.weights = np.linalg.inv((X.T.dot(X))).dot(X.T).dot(y)
        
    def predict(self, X_pred):
        w = self.weights
        return (X_pred.dot(w))

def RMSE(y, y_pred):
    return(np.sqrt(np.mean((y-y_pred)**2)))

def r2(y, y_pred):
    ss_res = np.sum((y-y_pred)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    return (1 - ss_res/ss_tot)


In [24]:
reg = OLS()
reg.fit(X, target)
pred = reg.predict(X)

In [33]:
r2(target, pred), RMSE(target, pred)

(0.19788283230970738, 3.9821924118803054)

In [26]:
ind = np.arange(n)
pred_loo = np.empty(n)
for i in range(n):
    loo_ind = np.delete(ind, i)
    reg.fit(X[loo_ind], target[loo_ind])
    pred_loo[i] = reg.predict(X[i])



In [27]:
r2(target, pred_loo), RMSE(target, pred_loo)

(-0.2585359870062167, 4.988111512698258)

In [30]:
ind_80 = np.where(target>80)
target_80, X_80 = target[ind_80], X[ind_80]
n_80 = target_80.shape[0]

In [31]:
pred_loo_80 = np.empty(n_80)
ind = np.arange(n_80)
reg = OLS()
for i in range(n_80):
    loo_ind = np.delete(ind, i)
    reg.fit(X_80[loo_ind], target_80[loo_ind])
    pred_loo_80[i] = reg.predict(X_80[i])

In [32]:
r2(target_80, pred_loo_80), RMSE(target_80, pred_loo_80)

(-0.4418268608312452, 3.6425372756360264)